<a href="https://www.kaggle.com/kamaljp/mannmachines-of-war?scriptVersionId=89502911" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### <a id="cont">Motive

Violence is last resort of the Incompetent... 
                               - Issac Asimov, Russian Born American Writer. 

Logic and mathematics are the languages this universe is programmed. When these two language fails, then Violence is taken as a last resort. Logic is 4 Dimensional. It has got Time as the 4th Dimension, where the leaders have to decide and act logically to avoid the last resort. They should avoid breaking the LOGIC. 

This dataset, and the analysis of it has to be a GLUE that makes the broken logic a whole. That is the way to resolve this conflict and establish peace. The analysis below shows 

1) [Where is the War happening?](#vis_1)

2) [How is the distribution of all equipments on last day?](#vis_2)

3) [What has changed on the last day?](#vis_3)

4) [How many machines are used in the War?](#vis_4)
    
5) [How many humans are used in the War?](#vis_5)
    
6) [How many additional humans are sighted the warzone each day?](#vis_6)
    
7) [How many additional equipments are sighted the warzone each day?](#vis_7)    

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import plotly.express as px
import plotly.graph_objects as go
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ukraine-russian-war/russia_losses_equipment.csv
/kaggle/input/2022-ukraine-russian-war/russia_losses_personnel.csv


In [2]:
equipment = pd.read_csv("/kaggle/input/2022-ukraine-russian-war/russia_losses_equipment.csv")
personnel = pd.read_csv("/kaggle/input/2022-ukraine-russian-war/russia_losses_personnel.csv")

In [3]:
equipment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   date                   9 non-null      object
 1   day                    9 non-null      int64 
 2   aircraft               9 non-null      int64 
 3   helicopter             9 non-null      int64 
 4   tank                   9 non-null      int64 
 5   APC                    9 non-null      int64 
 6   field artillery        9 non-null      int64 
 7   BUK                    9 non-null      int64 
 8   MRL                    9 non-null      int64 
 9   military auto          9 non-null      int64 
 10  fuel tank              9 non-null      int64 
 11  drone                  9 non-null      int64 
 12  naval ship             9 non-null      int64 
 13  anti-aircraft warfare  9 non-null      int64 
dtypes: int64(13), object(1)
memory usage: 1.1+ KB


In [4]:
personnel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        9 non-null      object 
 1   day         9 non-null      int64  
 2   personnel   9 non-null      int64  
 3   POW         2 non-null      float64
 4   *personnel  1 non-null      object 
dtypes: float64(1), int64(2), object(2)
memory usage: 488.0+ bytes


In [5]:
#Joining the data frames on the days columns to streamline the visualisation
manNmachine = pd.merge(left=equipment,right=personnel,left_on='day',right_on='day',how='left')

manNmachine.drop('date_y',axis=1,inplace=True)

In [6]:
#One detail that I missed is the interval of the days the data has been provided. 
#This detail can calculate the speed at which things are added
manNmachine["data_interval"] = manNmachine.day - manNmachine.day.shift(1)
manNmachine.loc[manNmachine.day == 3,"data_interval"] = 3

In [7]:
manNmachine['totalEquipment'] = manNmachine.iloc[:,3:-2].sum(axis=1)
manNmachine['Equipment_change_PDay'] = round((manNmachine.totalEquipment-manNmachine.totalEquipment.shift(1))/manNmachine.data_interval,0)
manNmachine['Human_change_PDay'] = round((manNmachine.personnel - manNmachine.personnel.shift(1))/manNmachine.data_interval,0)

In [8]:
manNmachine.iloc[0,-2] = round(manNmachine.iloc[0,-3]/manNmachine.iloc[0,-4],0)
manNmachine.iloc[0,-1] = round(manNmachine.iloc[0,-6]/manNmachine.iloc[0,-4],0)

In [9]:
manNmachine.head(2)

,date_x,day,aircraft,helicopter,tank,APC,field artillery,BUK,MRL,military auto,...,drone,naval ship,anti-aircraft warfare,personnel,POW,*personnel,data_interval,totalEquipment,Equipment_change_PDay,Human_change_PDay
0,2022-02-26,3,27,26,146,706,49,1,4,30,...,2,2,0,4300,NaN,NaN,3.0,5326.0,1775.0,NaN
1,2022-02-28,5,29,29,150,816,74,1,21,291,...,3,2,5,5300,NaN,NaN,2.0,6752.0,713.0,500.0


POW - Prisoner of War,

MRL - Multiple Rocket Launcher,

BUK - Buk Missile System,

APC - Armored Personnel Carrier,

drone: UAV - Unmanned Aerial Vehicle, RPA - Remotely Piloted Vehicle.

### Important. Each new record is accumulated data from previous days. The interval the data can be different, so the following features added

totalEquipment : Total equipment including all kinds of hardware involved in the War

Equipment_change_PDay : The change in equipment detected into the warzone per day

Human_change_PDay : The change number of humans sent to warzone per day

### What is getting added to the War?

In [10]:
def build_bar(dataset,column):
    fig = go.Figure()
    fig.add_trace(go.Bar(x=dataset.index,y=dataset[column],name=column))
    fig.update_layout(height=800,title_text='How many Russian '+ column + ' are spotted everyday?')
    return fig

### <a id="vis_4">How many machines are used in the War? 

In [11]:
build_bar(manNmachine,'totalEquipment')

[Back to Contents](#cont)

### <a id="vis_5"> How many Army, Navy and Air force men and women are involved

In [12]:
build_bar(manNmachine,'personnel')

[Back to Contents](#cont)

In [13]:
build_bar(manNmachine,'aircraft')

[Back to Contents](#cont)

### <a id="vis_3"> What has changed on the last day?

In [14]:
#The last change
manmachine = manNmachine.columns[3:]
Last_change = manNmachine.iloc[-1,3:-2] - manNmachine.iloc[-2,3:-2] 
px.bar(data_frame=Last_change,x=Last_change.index,y=Last_change.values)

[Back to Contents](#cont)

### <a id="vis_2"> How is the distribution of all equipments on last day?

In [15]:
#The last change
manmachine = manNmachine.columns[3:-4]
fire_col = go.Figure()
fire_col.add_trace(go.Scatter(x=manmachine,y=manNmachine.iloc[-1,3:-2]))

[Back to Contents](#cont)

In [16]:
pivot_mNm = manNmachine.iloc[:,2:]
pivot_mNm = pivot_mNm.T
pivot_mNm

,0,1,2,3,4,5,6,7,8
aircraft,27,29,29,30,30,33,39,44,46
helicopter,26,29,29,31,31,37,40,48,68
tank,146,150,198,211,217,251,269,285,290
APC,706,816,846,862,900,939,945,985,999
field artillery,49,74,77,85,90,105,105,109,117
BUK,1,1,1,1,1,1,1,1,1
MRL,4,21,24,40,42,50,50,50,50
military auto,30,291,305,355,374,404,409,447,454
fuel tank,60,60,60,60,60,60,60,60,60
drone,2,3,3,3,3,3,3,4,7


In [17]:
#Where the equipments are??
pivot_mNm.loc[:2,'type'] = 'Air'
pivot_mNm.loc[2:9,'type'] = 'Land'
pivot_mNm.loc['anti-aircraft warfare','type'] = 'Land'
pivot_mNm.loc['personnel','type'] = 'Land'
pivot_mNm.loc['drone','type'] = 'Air'
pivot_mNm.loc['naval ship','type'] = 'Sea'

### <a id="vis_1"> Where is the action happening?

In [18]:
theatre = pivot_mNm.groupby('type')[5].sum().reset_index()
px.bar(data_frame=theatre,x='type',y=5)

### <a id="vis_6"> How many additional humans are sighted the warzone each day?

In [19]:
build_bar(manNmachine,'Human_change_PDay')

[Back to Contents](#cont)

### <a id="vis_7"> How many additional equipments are sighted the warzone each day?

In [20]:
build_bar(manNmachine,'Equipment_change_PDay')

[Back to Contents](#cont)